Gradient Descent implementation

In [ ]:
from typing import Callable

def grad_descent(f: Callable, grad_f: Callable, step_fun: Callable[[int], float]):
  pass

Learning rate

Learning rate scheduling

One-dimensional search

In [ ]:
def one_dim_search(f: Callable[[float], float], left: float, right: float, eps: float):
  pass

Gradient descent using one-dimensional search

Gradient descent trajectory for quadratic functions

Number of iterations dependency on k, n